# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['台北市長柯文哲上午赴議會針對疫情紓困、無現金交易、老人安養等政策進行專案報告。針對疫情紓困，市府已提出9到12月的紓困措施，包括有稅籍者先延租再減租，娛樂稅也減半徵收7個月，若有業者到了年底還不行，可能要考慮轉行。', '\n', '\r\n柯文哲指出，2020年實質GDP成長率，全球經濟衰退５.5% 、台灣成長1.56%；2020年1至8月出口成長率，除了台灣正常1.5%，其餘主要國家均為負成長。這是因台灣出口資通訊產品，而台灣GDP成長率較高，這不是台灣厲害，而是別人衰退太多。', '\n', '\r\n柯文哲指出，北市109年1至6月營利事業銷售額年增率，批發零售業 、運輸倉儲業 、住宿餐飲業與藝術、娛樂及休閒服務在5、6月營業額年增率，衰退減幅均有少，呈V 型反轉，顯示疫情對經濟衝擊已漸緩和，但藝術娛樂休閒服務業沒有恢復，這些事業有傷到，要特別處理，包括延租減租，但到年底還不行怎麼辦？可能要考慮轉行。', '\n', '\r\n柯文哲指出，他估疫情還會持續1年半左右，今年耶誕節到明年過年前疫情還不會過去，北市災害準備金仍有3億元額度，市府還在規畫如何促進台北旅遊。', '\n', '\r\n柯文哲提到，今年10月12日為止，防疫旅館家數已達到73家，供可檢疫房間數5041間，完成接待超過2萬2500人， 防疫旅館是是台灣免於境外移入的戰略，未來居家檢疫隔離會嚴格執行，並嚴查日租套房，這些是阻絕國外疫情傳入台灣的方法。柯文哲對於近日引發爭議的跨年活動停辦的標準，柯說，未來有一種新常態叫風險管理，未來是充滿風險的年代。', '\n                    經濟部昨（23）日發布9月製造業生產指數123.91，年增11.32%，續創歷年單月新高，為連續第八個月正成長。經濟部統...                  ', '\n                    政府力推能源轉型，行政院副院長沈榮津今天表示，在台灣與歐洲具有經驗的開發商、系統商、設備商合作下，離岸風電已見初步成果，...                  ', '\n                    為防堵關鍵技術外流，經濟部投審會預告修訂「在大陸地區從事投資或技術合作許可辦法」第5條，針對授權或轉讓專門技術到中國，將...                  ', '\

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.617 seconds.


Prefix dict has been built successfully.


['台北市長 柯文哲 上午 赴 議會 針對 疫情 紓困 無 現金 交易 老人 安養 等 政策 進行 專案 報告 針對 疫情 紓困 市府 已 提出 到 月 的 紓困 措施 包括 有 稅籍 者 先延租 再 減租 娛樂稅 也 減半 徵收 個 月 若有 業者 到 了 年底 還 不行 可能 要 考慮 轉行 柯文哲 指出 年 實質 成長率 全球 經濟 衰退 台灣 成長 年 至 月 出口 成長率 除了 台灣 正常 其餘 主要 國家 均 為 負成長 這是 因 台灣 出口 資通訊 產品 而 台灣 成長率 較 高 這不 是 台灣 厲害 而是 別人 衰退 太多 柯文哲 指出 北市 年 至 月 營利 事業 銷售額 年增率 批發 零售業 運輸 倉儲業 住宿 餐飲業 與 藝術 娛樂 及 休閒 服務 在 月 營業額 年增率 衰退 減幅 均 有 少 呈 型 反轉 顯示 疫情 對 經濟 衝擊 已漸 緩和 但 藝術 娛樂 休閒 服務業 沒有 恢復 這些 事業 有 傷到 要 特別 處理 包括 延租 減租 但 到 年底 還 不行 怎麼辦 可能 要 考慮 轉行 柯文哲 指出 他估 疫情 還 會 持續 年 半 左右 今年 耶誕節 到 明年 過年 前 疫情 還 不會 過去 北市 災害 準備金 仍 有 億元 額度 市府 還在 規畫 如何 促進 台北 旅遊 柯文哲 提到 今年 月 日 為止 防疫 旅館 家數 已 達到 家 供 可 檢疫 房間數 間 完成 接待 超過 萬 人 防疫 旅館 是 是 台灣 免於 境外 移入 的 戰略 未來 居家 檢疫 隔離 會 嚴格 執行 並 嚴查 日 租套房 這些 是 阻絕 國外 疫情 傳入 台灣 的 方法 柯文哲 對於 近日 引發 爭議 的 跨年 活動 停辦 的 標準 柯 說 未來 有 一種 新常態 叫 風險 管理 未來 是 充滿 風險 的 年代 經濟部 昨 日 發布 月 製造業 生產 指數 年增 續創 歷年 單月 新高 為 連續 第八個 月 正成長 經濟部 統 政府 力推 能源 轉型 行政院 副院長 沈榮津 今天 表示 在 台灣 與 歐洲 具有 經驗 的 開發商 系統商 設備商 合作 下 離 岸風電 已見 初步 成果 為 防堵 關鍵 技術 外流 經濟部 投審會 預告 修訂 在 大陸 地區 從事 投資 或 技術 合作 許可 辦法 第 條 針對 授權 或 轉讓 專門 技術 到 中國 將 